In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
df=pd.read_excel('Test_set.xlsx')


In [3]:
type(df['Date_of_Journey'][0])

str

In [4]:
df=df.dropna()

In [5]:
def toDateTime(col):
    df[col]=pd.to_datetime(df[col], format='mixed')

In [6]:
for col in ['Dep_Time', 'Arrival_Time', 'Date_of_Journey']:
    toDateTime(col)

In [7]:
df["journey_day"]=df['Date_of_Journey'].dt.day
df["journey_month"]=df['Date_of_Journey'].dt.month
df["journey_year"]=df['Date_of_Journey'].dt.year

In [8]:
def extract_hour_min(df , col):
    df[col+"_hour"] = df[col].dt.hour
    df[col+"_minute"] = df[col].dt.minute

In [9]:
extract_hour_min(df , "Dep_Time")
extract_hour_min(df , "Arrival_Time")

In [10]:
df.drop(['Arrival_Time','Dep_Time'],axis=1,inplace=True)

In [11]:
def preprocess_duration(dur):
    if 'h' not in dur:
        dur = '0h' + ' ' + dur
    elif 'm' not in dur:
        dur = dur + ' ' +'0m'
        
    return dur
df['Duration']=df['Duration'].apply(preprocess_duration)
df['Duration_hours'] = df['Duration'].apply(lambda x : int(x.split(' ')[0][0:-1]))
df['Duration_mins'] = df['Duration'].apply(lambda x : int(x.split(' ')[1][0:-1]))

In [12]:
df['Duration_total_mins'] = df['Duration'].str.replace('h' ,"*60").str.replace(' ' , '+').str.replace('m' , "*1").apply(eval)

In [13]:
df['Destination'].replace('New Delhi' , 'Delhi' , inplace=True)

In [14]:
for source in df['Source'].unique():
    df['Source_'+source] = df['Source'].apply(lambda x : 1 if x==source else 0)


In [15]:
import pickle

In [16]:
with open('encodings.pkl', 'rb') as file:
    unique_source = pickle.load(file)
    airline_dict = pickle.load(file)
    dest_dict = pickle.load(file)
    stops = pickle.load(file)

In [17]:
airline_dict

{'Trujet': 0,
 'SpiceJet': 1,
 'Air Asia': 2,
 'IndiGo': 3,
 'GoAir': 4,
 'Vistara': 5,
 'Vistara Premium economy': 6,
 'Air India': 7,
 'Multiple carriers': 8,
 'Multiple carriers Premium economy': 9,
 'Jet Airways': 10,
 'Jet Airways Business': 11}

In [18]:
unique_source

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [19]:
dest_dict

{'Kolkata': 0, 'Hyderabad': 1, 'Delhi': 2, 'Banglore': 3, 'Cochin': 4}

In [20]:
stops

{'non-stop': 0, '2 stops': 2, '1 stop': 1, '3 stops': 3, '4 stops': 4}

In [21]:
for source in unique_source:
    df['Source_'+source] = df['Source'].apply(lambda x : 1 if x==source else 0)

In [22]:
df['Airline'] = df['Airline'].map(airline_dict)

In [23]:
df['Destination']=df['Destination'].map(dest_dict)

In [24]:
df['Total_Stops'] = df['Total_Stops'].map(stops)

In [25]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Additional_Info,journey_day,journey_month,...,Arrival_Time_hour,Arrival_Time_minute,Duration_hours,Duration_mins,Duration_total_mins,Source_Delhi,Source_Kolkata,Source_Banglore,Source_Mumbai,Source_Chennai
0,10,2019-06-06,Delhi,4,DEL → BOM → COK,10h 55m,1,No info,6,6,...,4,25,10,55,655,1,0,0,0,0
1,3,2019-12-05,Kolkata,3,CCU → MAA → BLR,4h 0m,1,No info,5,12,...,10,20,4,0,240,0,1,0,0,0
2,10,2019-05-21,Delhi,4,DEL → BOM → COK,23h 45m,1,In-flight meal not included,21,5,...,19,0,23,45,1425,1,0,0,0,0
3,8,2019-05-21,Delhi,4,DEL → BOM → COK,13h 0m,1,No info,21,5,...,21,0,13,0,780,1,0,0,0,0
4,2,2019-06-24,Banglore,2,BLR → DEL,2h 50m,0,No info,24,6,...,2,45,2,50,170,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,7,2019-06-06,Kolkata,3,CCU → DEL → BLR,23h 55m,1,No info,6,6,...,20,25,23,55,1435,0,1,0,0,0
2667,3,2019-03-27,Kolkata,3,CCU → BLR,2h 35m,0,No info,27,3,...,16,55,2,35,155,0,1,0,0,0
2668,10,2019-06-03,Delhi,4,DEL → BOM → COK,6h 35m,1,No info,3,6,...,4,25,6,35,395,1,0,0,0,0
2669,7,2019-06-03,Delhi,4,DEL → BOM → COK,15h 15m,1,No info,3,6,...,19,15,15,15,915,1,0,0,0,0


In [26]:
df.drop(['Date_of_Journey','Source','Route','Additional_Info','journey_year','Duration','Duration_total_mins'],axis=1,inplace=True)

In [27]:
df

,Airline,Destination,Total_Stops,journey_day,journey_month,Dep_Time_hour,Dep_Time_minute,Arrival_Time_hour,Arrival_Time_minute,Duration_hours,Duration_mins,Source_Delhi,Source_Kolkata,Source_Banglore,Source_Mumbai,Source_Chennai
0,10,4,1,6,6,17,30,4,25,10,55,1,0,0,0,0
1,3,3,1,5,12,6,20,10,20,4,0,0,1,0,0,0
2,10,4,1,21,5,19,15,19,0,23,45,1,0,0,0,0
3,8,4,1,21,5,8,0,21,0,13,0,1,0,0,0,0
4,2,2,0,24,6,23,55,2,45,2,50,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,7,3,1,6,6,20,30,20,25,23,55,0,1,0,0,0
2667,3,3,0,27,3,14,20,16,55,2,35,0,1,0,0,0
2668,10,4,1,3,6,21,50,4,25,6,35,1,0,0,0,0
2669,7,4,1,3,6,4,0,19,15,15,15,1,0,0,0,0


In [28]:
with open('rf_random.pkl','rb') as file:
    model=pickle.load(file)

In [32]:
with open('col_seq','rb') as file:
    colseq=pickle.load(file)

In [33]:
df=df[colseq]

In [34]:
y=model.predict(df)

In [35]:
y

array([10925.51766667,  4226.        , 15282.2875    , ...,
       14426.33266667,  9674.69      ,  7528.62      ])